# Notebook inicial para o projeto final de Aprendizagem de Máquina.

1. Escolher o tema e os dados.
2. Definir bem o workflow/pipeline de dados, como vamos realizar o nosso trabalho.
3. Botar a mão na massa.

## Endpoints que iremos utilizar:

* Link com todos os endpoints: https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx

* ListarProposicoes: https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ListarProposicoes

* ListarProposicoesVotadasEmPlenario: https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ListarProposicoesVotadasEmPlenario

* ObterVotacaoProposicaoPorID: https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicaoPorID

* ObterVotacaoProposicao: https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ObterProposicaoPorID

## Pipeline de uso dos endpoints:

1. Pegar as proposições e colocar num csv. (Definir os anos. Vamos focar em Projetos de Lei.)
2. Criar nova(s) coluna(s) em relação ao fato de aquela proposição ter sido votada
3. Decidir se filtramos só as que foram votadas ou se vamos utilizar o fato de ser votada ou não como uma feature.
4. Ver se há necessidade de criar novas colunas.
5. Adicionar os resultados das votações.
6. Pensar nos próximos passos.


In [1]:
import requests, time, os
from datetime import datetime
import xml.etree.ElementTree as ET
import xml_to_df
import pandas as pd

In [2]:
df = pd.read_csv('data/ListaProposicoes_2013_2024.csv')

#df = pd.read_csv('data/ListaProposicoesVotadasPlenario_2013_2024.csv')

In [5]:
url = 'https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ObterProposicaoPorID'
responses = {}
dfs = {}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

for id_prop in df['id'].unique():
    params = {
        'idProp': id_prop 
    }

    response = xml_to_df.get_data(url, params, headers)

    if response.status_code != 200:
        print(f'erro ao extrair dados de id {id_prop}')
        continue
    
    df_id = xml_to_df.data_to_df(response, ObterProposicaoPorID='porId')
    responses[id_prop] = response
    dfs[id_prop] = df_id
    print(f'dados de id {id_prop} extraidos com sucesso')
    
print(f'dados de id extraidos com sucesso')

merged_df = pd.concat(dfs.values(), ignore_index=True)
xml_to_df.save_csv(df, "ListaProposicoes_2013_2024.csv")


dados de id 2462403 extraidos com sucesso
dados de id 2462322 extraidos com sucesso
dados de id 2462316 extraidos com sucesso
dados de id 2462314 extraidos com sucesso
dados de id 2462306 extraidos com sucesso
dados de id 2462304 extraidos com sucesso
dados de id 2462305 extraidos com sucesso
dados de id 2462276 extraidos com sucesso
dados de id 2462271 extraidos com sucesso
dados de id 2462274 extraidos com sucesso
dados de id 2462270 extraidos com sucesso
dados de id 2462264 extraidos com sucesso
dados de id 2462229 extraidos com sucesso
dados de id 2462207 extraidos com sucesso
dados de id 2462206 extraidos com sucesso
dados de id 2462205 extraidos com sucesso
dados de id 2462202 extraidos com sucesso
dados de id 2462191 extraidos com sucesso
dados de id 2462146 extraidos com sucesso
dados de id 2462145 extraidos com sucesso
dados de id 2462144 extraidos com sucesso
dados de id 2462143 extraidos com sucesso
dados de id 2462142 extraidos com sucesso
dados de id 2462129 extraidos com 

In [7]:
xml_to_df.save_csv(merged_df, "Proposicoes_2013_2024.csv")

In [7]:

%pip install zeep
import zeep
import pandas as pd
from zeep import Client
from zeep.transports import Transport
import requests
import time

# Definir a URL do WSDL
wsdl = 'https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx?WSDL'

# Configurar a sessão com headers apropriados
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
})

# Criar um transport com a sessão
transport = Transport(session=session)

# Criar o cliente Zeep
client = zeep.Client(wsdl=wsdl, transport=transport)

# Função para obter dados da proposição por ID
def obter_proposicao_por_id(id_prop):
    try:
        response = client.service.ObterProposicaoPorID(idProp=id_prop)
        return response
    except Exception as e:
        print(f'Erro ao obter dados para id {id_prop}: {e}')
        return None

# Inicializar dicionários para armazenar respostas e DataFrames
responses = {}
dfs = {}

# Supondo que 'df' seja o seu DataFrame original com a coluna 'id'
for id_prop in df['id'].unique()[:3]:
    print(f'Processando id: {id_prop}')
    response = obter_proposicao_por_id(id_prop)
    
    if response is None:
        print(f'Erro ao extrair dados de id {id_prop}')
        continue
    
    # Converter a resposta para um dicionário
    data_dict = zeep.helpers.serialize_object(response, target_cls=dict)
    
    if not data_dict:
        print(f'Nenhum dado retornado para id {id_prop}')
        continue
    
    # Inspecionar a estrutura do dicionário
    # print(data_dict)  # Descomente para ver a estrutura
    
    # Verificar se os campos esperados estão presentes
    if 'Proposicao' in data_dict:
        proposicao = data_dict['Proposicao']
        
        # Transformar em DataFrame
        df_id = pd.json_normalize(proposicao)
        
        responses[id_prop] = proposicao
        dfs[id_prop] = df_id
    else:
        print(f'Estrutura inesperada na resposta para id {id_prop}')
    
    # Pausa para evitar sobrecarregar o servidor
    time.sleep(0.5)

print('Dados de id extraídos com sucesso')

# Verificar se há DataFrames para concatenar
if dfs:
    merged_df = pd.concat(dfs.values(), ignore_index=True)
    merged_df.to_csv("ListaProposicoes_2013_2024.csv", index=False, encoding='utf-8')
    print("Arquivo CSV salvo com sucesso.")
else:
    print("Nenhum dado foi extraído para salvar.")


  Using cached zeep-4.3.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached isodate-0.7.2-py3-none-any.whl.metadata (11 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached requests_file-2.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached zeep-4.3.0-py3-none-any.whl (101 kB)
Using cached isodate-0.7.2-py3-none-any.whl (22 kB)
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/3.8 MB 728.2 kB/s eta 0:00:05
   -------- ------------------------------- 0.8/3.8 MB 907.1 kB/s eta 0:00:04
   ---------- ----------------------------- 1.0/3.8 MB 967.3 kB/s eta 0:00:03
   ------------- -------------------------- 1.3/3.8 MB 1.1 MB/s eta 0:00:03
   ---------------- -----------------

Forcing http:address location to HTTPS
Forcing http:address location to HTTPS
C:\Users\kamil\AppData\Local\Temp\ipykernel_3280\3260890635.py:49: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not data_dict:


Processando id: 2462403
Estrutura inesperada na resposta para id 2462403
Processando id: 2462322
Estrutura inesperada na resposta para id 2462322
Processando id: 2462316
Estrutura inesperada na resposta para id 2462316
Dados de id extraídos com sucesso
Nenhum dado foi extraído para salvar.


In [26]:
root = ET.fromstring(response.text)


ParseError: mismatched tag: line 71, column 94 (<string>)

In [24]:
response

<Response [200]>

In [4]:
responses = {}
dfs = {}

for ano in range(2013,2025):
    params = {
    'sigla': 'PL',  # Projeto de Lei
    'numero': '',
    'ano': ano,
    'datApresentacaoIni': f'01/01/{ano}',
    'datApresentacaoFim': f'31/12/{ano}',
    'idTipoAutor': '',
    'parteNomeAutor': '',
    'siglaPartidoAutor': '',
    'siglaUFAutor': '',
    'generoAutor': '',
    'codEstado': '',
    'codOrgaoEstado': '',
    'emTramitacao': ''
}
    
    response = xml_to_df.get_data(url, params, headers)
    df = xml_to_df.data_to_df(response)
    responses[ano] = response
    dfs[ano] = df
    print(f'Dados de {ano} extraídos com sucesso!')

merged_df = pd.concat(dfs.values(), ignore_index=True)
xml_to_df.save_csv(df, "ListaProposicoes_2013_2024.csv")


Dados de 2013 extraídos com sucesso!
Dados de 2014 extraídos com sucesso!
Dados de 2015 extraídos com sucesso!
Dados de 2016 extraídos com sucesso!
Dados de 2017 extraídos com sucesso!
Dados de 2018 extraídos com sucesso!
Dados de 2019 extraídos com sucesso!
Dados de 2020 extraídos com sucesso!
Dados de 2021 extraídos com sucesso!
Dados de 2022 extraídos com sucesso!
Dados de 2023 extraídos com sucesso!
Dados de 2024 extraídos com sucesso!


In [5]:
merged_df = pd.concat(dfs.values(), ignore_index=True)
xml_to_df.save_csv(df, "ListaProposicoes_2013_2024.csv")

In [7]:
merged_df.ano.unique()

array(['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022', '2023', '2024'], dtype=object)

In [8]:
merged_df.columns

Index(['id', 'nome', 'tipoProposicao_id', 'tipoProposicao_sigla',
       'tipoProposicao_nome', 'numero', 'ano', 'orgaoNumerador_id',
       'orgaoNumerador_sigla', 'orgaoNumerador_nome', 'datApresentacao',
       'txtEmenta', 'txtExplicacaoEmenta', 'regime_codRegime',
       'regime_txtRegime', 'apreciacao_id', 'apreciacao_txtApreciacao',
       'autor1_txtNomeAutor', 'autor1_idecadastro', 'autor1_codPartido',
       'autor1_txtSiglaPartido', 'autor1_txtSiglaUF', 'qtdAutores',
       'ultimoDespacho_datDespacho', 'ultimoDespacho_txtDespacho',
       'situacao_id', 'situacao_descricao', 'situacao_orgao',
       'situacao_principal', 'indGenero', 'qtdOrgaosComEstado'],
      dtype='object')

In [7]:
dfs[2023]

,id,nome,tipoProposicao_id,tipoProposicao_sigla,tipoProposicao_nome,numero,ano,orgaoNumerador_id,orgaoNumerador_sigla,orgaoNumerador_nome,...,autor1_txtSiglaUF,qtdAutores,ultimoDespacho_datDespacho,ultimoDespacho_txtDespacho,situacao_id,situacao_descricao,situacao_orgao,situacao_principal,indGenero,qtdOrgaosComEstado
0,2416789,PL 5816/2023,139,PL,Projeto de Lei,5816,2023,180,PLEN,Plenário,...,PE,1,06/02/2024 11:25:00,Apense-se à(ao) PL-5751/2023.\nProposição Suje...,925,Tramitando em Conjunto,\n,\n,o,1
1,2416781,PL 4149/2023,139,PL,Projeto de Lei,4149,2023,180,PLEN,Plenário,...,RO,1,06/02/2024 11:11:00,Às Comissões de \nEducação; \nDefesa dos Direi...,915,Aguardando Parecer,\n,\n,o,1
2,2416780,PL 4150/2023,139,PL,Projeto de Lei,4150,2023,180,PLEN,Plenário,...,RO,1,06/02/2024 11:11:00,Às Comissões de \nCultura e \nConstituição e J...,907,Aguardando Designação de Relator(a),\n,\n,o,1
3,2416758,PL 1593/2023,139,PL,Projeto de Lei,1593,2023,180,PLEN,Plenário,...,AP,1,06/02/2024 11:11:00,Apense-se à(ao) PL-3462/2023. Em decorrência d...,925,Tramitando em Conjunto,\n,\n,o,1
4,2416762,PL 3386/2023,139,PL,Projeto de Lei,3386,2023,180,PLEN,Plenário,...,MT,1,06/02/2024 11:11:00,Às Comissões de \nViação e Transportes; \nCult...,915,Aguardando Parecer,\n,\n,o,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5366,2345643,PL 12/2023,139,PL,Projeto de Lei,12,2023,180,PLEN,Plenário,...,MA,1,07/03/2023 16:33:00,Apense-se à(ao) PL-3/2023. Proposição Sujeita ...,923,Arquivada,\n,\n,o,1
5367,2345646,PL 14/2023,139,PL,Projeto de Lei,14,2023,180,PLEN,Plenário,...,PE,1,07/03/2023 16:33:00,Apense-se à(ao) PL-3/2023. Proposição Sujeita ...,923,Arquivada,\n,\n,o,1
5368,2345625,PL 3/2023,139,PL,Projeto de Lei,3,2023,180,PLEN,Plenário,...,RS,27,29/11/2023,SUBSTITUTIVO DO SENADO FEDERAL \nÀs Comissões ...,1140,Transformado em Norma Jurídica,\n,\n,o,1
5369,2345627,PL 4/2023,139,PL,Projeto de Lei,4,2023,180,PLEN,Plenário,...,MG,2,07/03/2023 16:33:00,Apense-se à(ao) PL-3/2023. Proposição Sujeita ...,923,Arquivada,\n,\n,o,1


## Proposicoes Votadas Em Plenario

In [15]:
# pvp = proposicoes_votadas_plenario

url_pvp = 'https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ListarProposicoesVotadasEmPlenario'
ano = 2020
params = {
    'ano': ano,
    'tipo': 'PL'
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url_pvp, params=params, headers=headers)

In [16]:
response

<Response [404]>

In [17]:
responses_pvp = {}
dfs_pvp = {}

for ano in range(2013,2025):
    if ano == 2020: ## 2020 não tem dados 
        continue

    params = {
    'ano': ano,
    'tipo': 'PL'
}
    
    response = xml_to_df.get_data(url_pvp, params, headers)
    df = xml_to_df.data_to_df(response)
    responses_pvp[ano] = response
    dfs_pvp[ano] = df
    print(f'Dados de {ano} extraídos com sucesso!')

Dados de 2013 extraídos com sucesso!
Dados de 2014 extraídos com sucesso!
Dados de 2015 extraídos com sucesso!
Dados de 2016 extraídos com sucesso!
Dados de 2017 extraídos com sucesso!
Dados de 2018 extraídos com sucesso!
Dados de 2019 extraídos com sucesso!
Dados de 2021 extraídos com sucesso!
Dados de 2022 extraídos com sucesso!
Dados de 2023 extraídos com sucesso!
Dados de 2024 extraídos com sucesso!


In [18]:
merged_df_pvp = pd.concat(dfs_pvp.values(), ignore_index=True)
xml_to_df.save_csv(merged_df_pvp, "ListaProposicoesVotadasPlenario_2013_2024.csv")

## Obter Votações Por ID (Refazer para salvar os dados de forma correta)

In [35]:
# vpi = votacoes_por_id

url_vpi = 'https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicaoPorID'
id = 1672576
params = {
    'idProposicao': id
}

In [50]:
import xml.etree.ElementTree as ET
import pandas as pd

def parse_element(element, parent_tag=''):
    """
    Recursively parse an XML element and return a dictionary of its data.
    """
    data = {}
    for child in element:
        tag = f"{parent_tag}_{child.tag}" if parent_tag else child.tag
        if len(child) > 0:  # If the child has sub-elements, recurse
            data.update(parse_element(child, tag))
        else:
            data[tag] = child.text
    return data

def data_to_df(response):
    # Parse XML
    root = ET.fromstring(response.text)
    proposicoes = root.findall('proposicao')

    # Colocar dados em listas de dicionários
    lista_dados = []

    for p in proposicoes:
        dados = parse_element(p)
        lista_dados.append(dados)

    # Criar DataFrame
    df = pd.DataFrame(lista_dados)
    return df

In [40]:
responses_vpi = {}
dfs_vpi = {}

for id in merged_df_pvp.codProposicao.unique():
    params = {
        'idProposicao': id
    }
    
    try:
        response = xml_to_df.get_data(url_vpi, params, headers)
        
        if response.status_code != 200:
            print(f'Erro ao extrair dados de votação do PL de id = {id}')
            continue
        
        df = xml_to_df.data_to_df(response)
        responses_vpi[id] = response
        dfs_vpi[id] = df
        print(f'Dados de votação do PL de id = {id} extraídos com sucesso!')
    
    except Exception as e:
        print(f'Erro ao processar dados de votação do PL de id = {id}: {e}')

Dados de votação do PL de id = 271219 extraídos com sucesso!
Dados de votação do PL de id = 302638 extraídos com sucesso!
Dados de votação do PL de id = 334331 extraídos com sucesso!
Dados de votação do PL de id = 343930 extraídos com sucesso!
Dados de votação do PL de id = 347880 extraídos com sucesso!
Dados de votação do PL de id = 483808 extraídos com sucesso!
Dados de votação do PL de id = 537784 extraídos com sucesso!
Dados de votação do PL de id = 556261 extraídos com sucesso!
Dados de votação do PL de id = 564113 extraídos com sucesso!
Dados de votação do PL de id = 586411 extraídos com sucesso!
Dados de votação do PL de id = 592935 extraídos com sucesso!
Dados de votação do PL de id = 19319 extraídos com sucesso!
Dados de votação do PL de id = 490116 extraídos com sucesso!
Dados de votação do PL de id = 552295 extraídos com sucesso!
Dados de votação do PL de id = 600255 extraídos com sucesso!
Dados de votação do PL de id = 601914 extraídos com sucesso!
Dados de votação do PL de

In [ ]:
response_test = responses_vpi['271219']
root = ET.fromstring(response_test.text)
xml_text = ET.tostring(root, encoding='unicode')
print(xml_text)

In [98]:
for votacao in root.findall(".//Votacao"):
    resumo = votacao.get("Resumo")
    data = votacao.get("Data")
    hora = votacao.get("Hora")
    obj_votacao = votacao.get("ObjVotacao")
    nome_proposicao = votacao.get("nomeProposicao")

    print(f"Resumo: {resumo}, Data: {data}, Hora: {hora}, Objeto: {obj_votacao}, Proposição: {nome_proposicao}")
    
    # Iterar sobre orientações de bancadas
    for bancada in votacao.findall(".//bancada"):
        sigla = bancada.get("Sigla")
        orientacao = bancada.get("orientacao")
        print(f"Bancada: {sigla}, Orientação: {orientacao.strip()}")

    # Iterar sobre os votos dos deputados
    for deputado in votacao.findall(".//Deputado"):
        nome = deputado.get("Nome")
        partido = deputado.get("Partido").strip()
        uf = deputado.get("UF")
        voto = deputado.get("Voto").strip()
        print(f"Deputado: {nome}, Partido: {partido}, UF: {uf}, Voto: {voto}")

Resumo: Mantido o texto. Sim: 268; não: 62; abstenção: 5; total: 335., Data: 9/7/2013, Hora: 23:02, Objeto: DVS - DEM - § 2º DO ART. 23 DO SUBSTITUTIVO DO SENADO FEDERAL, Proposição: PL 4529/2004
Bancada: PT, Orientação: Obstrução
Bancada: PMDB, Orientação: Obstrução
Bancada: PSDB, Orientação: Sim
Bancada: PSD, Orientação: Liberado
Bancada: PrPtdobPrpPhsPslPrtb, Orientação: Obstrução
Bancada: PP, Orientação: Liberado
Bancada: DEM, Orientação: Não
Bancada: PSB, Orientação: Sim
Bancada: PDT, Orientação: Sim
Bancada: PTB, Orientação: Obstrução
Bancada: PSC, Orientação: Sim
Bancada: PCdoB, Orientação: Sim
Bancada: PPS, Orientação: Não
Bancada: PV, Orientação: Sim
Bancada: PRB, Orientação: Sim
Bancada: PSOL, Orientação: Sim
Bancada: Repr.PMN, Orientação: Liberado
Bancada: Minoria, Orientação: Liberado
Deputado: Edio Lopes, Partido: PMDB, UF: RR, Voto: Sim
Deputado: Chico das Verduras, Partido: PRP, UF: RR, Voto: Não
Deputado: Chico das Verduras, Partido: PRP, UF: RR, Voto: Não
Deputado: Luc

In [96]:
print(xml_text.find('proposicao'))
print(root.find('.//proposicao'))

1
None


In [83]:
data_to_df(response_test)

Collected data: []
No data to create DataFrame.


""


In [41]:
merged_df_vpi = pd.concat(dfs_vpi.values(), ignore_index=True)
xml_to_df.save_csv(merged_df_vpi, "VotacoesProposicoes_2013_2024.csv")

In [42]:
dfs_vpi

{'271219': Empty DataFrame
 Columns: []
 Index: [],
 '302638': Empty DataFrame
 Columns: []
 Index: [],
 '334331': Empty DataFrame
 Columns: []
 Index: [],
 '343930': Empty DataFrame
 Columns: []
 Index: [],
 '347880': Empty DataFrame
 Columns: []
 Index: [],
 '483808': Empty DataFrame
 Columns: []
 Index: [],
 '537784': Empty DataFrame
 Columns: []
 Index: [],
 '556261': Empty DataFrame
 Columns: []
 Index: [],
 '564113': Empty DataFrame
 Columns: []
 Index: [],
 '586411': Empty DataFrame
 Columns: []
 Index: [],
 '592935': Empty DataFrame
 Columns: []
 Index: [],
 '19319': Empty DataFrame
 Columns: []
 Index: [],
 '490116': Empty DataFrame
 Columns: []
 Index: [],
 '552295': Empty DataFrame
 Columns: []
 Index: [],
 '600255': Empty DataFrame
 Columns: []
 Index: [],
 '601914': Empty DataFrame
 Columns: []
 Index: [],
 '619150': Empty DataFrame
 Columns: []
 Index: [],
 '18180': Empty DataFrame
 Columns: []
 Index: [],
 '18216': Empty DataFrame
 Columns: []
 Index: [],
 '267841': Empt